# Life 101

In this notebook, we create a simple analogy between Wikispeedia game and "Life". We found out that some pieces of advice extracted from the game are also applicable in real life and we can learn from them!

# Data Preparation

Please add folders `wikispeedia_articles_html`, `wikispeedia_articles_plaintext`, `wikispeedia_paths-and-graph` from the orignal dataset.

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import src.utils as utils
import plotly.express as px
import plotly.graph_objects as go



# Automatically reload the module in case of changes
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
data_path = "./data/wikispeedia_paths-and-graph/"

# Load finished paths
finished_paths = utils.load_dataframe(data_path + "paths_finished.tsv", columns=["hashedIpAddress", "timestamp", "durationInSec", "path", "rating"])

# Load unfinished paths
unfinished_paths = utils.load_dataframe(data_path + "paths_unfinished.tsv", columns=["hashedIpAddress", "timestamp", "durationInSec", "path", "target", "type"])

# Load categories
categories = utils.load_dataframe(data_path + "categories.tsv", columns=["page", "category"])

# Load links
links_df = utils.load_dataframe(data_path + "links.tsv", columns=["source", "target"])

In [20]:
# Data cleaning and preprocessing
categories_dict = utils.manage_categories(categories.copy())
finished_paths_df = utils.manage_paths(finished_paths.copy(), categories_dict.copy())
unfinished_paths_df = utils.manage_paths(unfinished_paths.copy(), categories_dict.copy())
links_dict = utils.manage_links(links_df.copy())

#create graph and page ranks
graph = utils.create_graph(links_df.copy())
ranks = utils.page_rank(graph)

# Save the cleaned dataframes
finished_paths_df.to_csv("./data/clean_data/clean_finished_paths.csv")
unfinished_paths_df.to_csv("./data/clean_data/clean_unfinished_paths.csv")

# 1. Don't trust LLMs blindly!

We will compare people performance to LLM performance by considering the most taken paths by people and launching them on a 4bit quantized version of Qwen-3b. A fairly small LLM that could run on our machines/Colab in a reasonable time. We run the same task from source to target 100 times using the LLM. This allows us to have statistically relevant paths especially that the LLM is sometimes lost and not able to finish the paths everytime. The file that generates the LLM results is /utils/LLM.py. Please note that CUDA is required for this file to run. (It may not generate the exact same paths but they will still have the same behaviour statistically.)

## Prompting strategy
In order to get the model to work, we did prompt engineering trying multiple instructions for the model. The overall idea of the prompt is that we give the model only the target and the list of possible links at a certain step and we ask it about the best choice in order to get closer to the target. If the answer is not in the list, we keep the context of the conversation and we tell the model that it gave us an article not in the list and we ask it to correct itself. If it does, then we continue, otherwise, the path is considered aborted.
If we have an answer, we restart with a completely new context using a similar prompt.

## Comparison strategy
As we get multiple samples of answers (100 in our case) from the LLM for each source-target pair, we get an estimate of the distribution of lengths of the paths given by the LLM. Similarly, we have multiple samples of player paths, so we cannot only compare the means of the path lengths but also do a t-test similarity test to know if the distributions are similar or not.

In [21]:
# Group the finished paths DataFrame by "source" and "target" columns, then count occurrences in each group
top_paths = finished_paths_df.groupby(["source", "target"])["hashedIpAddress"].count().rename("count")

# Sort the grouped data by the count in descending order to get the most frequent paths
top_paths = top_paths.sort_values(ascending=False).reset_index().head(10)
print("Top 10 paths: ")
print(top_paths)

# Extract the "source" column values of the top paths as a list
sources = top_paths['source'].tolist()

# Extract the "target" column values of the top paths as a list
targets = top_paths['target'].tolist()

# Get the LLM paths
# uncomment the following lines to run the LLM generation (CUDA is required) and comment the next one
# import src.LLM as LLM
# llm_paths = LLM.LLM(sources, targets, links_dict)
llm_paths = utils.read_llm_paths("./data/llm_paths.json")

Top 10 paths: 
     source             target  count
0  Asteroid             Viking   1043
1     Brain          Telephone   1040
2   Theatre              Zebra    905
3   Pyramid               Bean    642
4    Batman               Wood    148
5      Bird  Great_white_shark    138
6    Batman      The_Holocaust    119
7      Bird       Adolf_Hitler    107
8      Beer                Sun     99
9    Batman             Banana     69


In [22]:
# Plot LLM and players mean lengths and errors
utils.plot_llms_vs_players(sources, targets, finished_paths_df, llm_paths)
p_values = utils.tstats_pvalues(sources, targets, finished_paths_df, llm_paths)
utils.plot_tsatistics(sources, targets, p_values)


## Statistical analysis

The first plot shows that the LLM's performance is worse in all the source-target pairs. However, we are not confident that it's better in all of them. (For example, the players mean falls into the standard error of the LLM mean for the Brain-Telephone paths). This is why we also calculate the t-test p-values. We define our null hypothesis as "LLMs have larger average path length than human players". We can see that for all the source-target pairs, the p-values are much larger than 0.05 so we cannot reject the null hypothesis. As the p-values are very high, it indicates that our observation are not so unlikely to have occurred by chance.

Not only LLMs are not as good as humans but also they tried to cheat in the game. In fact, when we look at the number of games finished by the LLMs, we can see that for the paths they struggled on, they finished less games than for the easy paths. (example: the 3 source-target pairs with the highest mean lengths are : Bird -> Great white Shark, Theatre -> Zebra and Pyramid -> Bean they are also the 3 source-target pairs with the lowest number of finished paths.)
This means that they twice tried to give us an answer that is not in their list of possibilities so they did not follow the task.

In [23]:
number_of_finished_paths = {
    "Source_Target": [f"{source} -> {target}" for source, target in zip(sources, targets)],
    "Number_of_LLM_paths": [len(llm_paths[f"{source}_{target}"]) for source, target in zip(sources, targets)]
}
fig = px.bar(number_of_finished_paths, x="Source_Target", y="Number_of_LLM_paths", 
             title="Number of LLM paths finished out of 100 for each source-target pair")

fig.update_layout(
    xaxis_title="Source -> Target",
    yaxis_title="Number of LLM paths",
    xaxis_tickangle=45
)
fig.show()

## What is the LLM's strategy?
We studied the LLM strategy to know why they don't get as good performance as real players. We ran page rank algorithm on the graph of links and plotted the average ranks of nodes for the most common path lengths for players and LLMs on each source-target pair. We can see that players tend to go up quickly to a "Hub" and then go down to the target. The LLMs on the other hand don't have a clear pattern in their paths. This confirms to us that they are not good on all tasks and also that they are not very consistent. However, we observed that when they use the "Hub" strategy they are as god as humans (example: Batman -> The Holocaust path).

In [24]:
utils.plot_llm_vs_players_strategies(sources, targets, finished_paths_df, llm_paths, ranks)

## Take away about LLMs
LLMs are a very good tool that could make our lives easier in many ways. However, one should not trust them blindly. They can give us inaccurate or even wrong answers. We can ask them for help and advice but they are still far from human strategic thinking and way of reasoning.

# 2. Links coordinates analysis

We obtained the x and y coordinates of the links by using Selenium. It enables us to open the html file of the articles in a simulated browser and then select the links we are interested in. We set the window size of the browser to 1920x1080 because it is the most common. 
Also, when the next article in path is accessible via several links, we stored the coordinates of all these links.

We run this algorithm on each path of the 50 most popular source-target peers.

The code doing that can be found in the `src/links_coordinates.py` file.
 
The data collected this way is stored in the files `data/links_coordinates_optimal.csv` for the links coordinates of the optimal paths, `data/links_coordinates_finished.csv` for the links coordinates of the finished paths and `data/links_coordinates_unfinished.csv` for the links coordinates of the unfinished paths.

Let's vizualize where people click!

In [25]:
links_coords_optimal = pd.read_csv("data/links_coordinates_optimal.csv")
links_coords_finished = pd.read_csv("data/links_coordinates_finished.csv")
links_coords_unfinished = pd.read_csv("data/links_coordinates_unfinished.csv")

In [26]:
def plot_coordinates_distribution(links_coords_data, title):
    all_links_coords = []
    for links_coords in links_coords_data["links_coords"].apply(eval).values:
        all_links_coords.extend(links_coords)

    x_coords = [coord[0] for coord in all_links_coords]
    y_coords = [coord[1] for coord in all_links_coords]

    fig = px.density_contour(
        x=x_coords, y=y_coords, 
        title=title, 
        labels={'x': 'X Coordinates', 'y': 'Y Coordinates'},
    )
    fig.update_traces(contours_coloring="fill", colorscale="Reds")
    fig.update_yaxes(range=[4000, 0])
    fig.update_xaxes(range=[0, 1920])
    fig.update_layout(
        height=600,
    )
    fig.show()


In [27]:
plot_coordinates_distribution(links_coords_optimal, "Links coordinates of the optimal paths")

In [28]:
plot_coordinates_distribution(links_coords_unfinished, "Links coordinates of the unfinished paths")

In [29]:
plot_coordinates_distribution(links_coords_finished, "Links coordinates of the finished paths")

As we can see, we, as humans, are lazy. Indeed most of the links clicked are at the very top of the articles, probably in the first article while the best links can be found at almost any coordinates in the article as we can see in the first plot.

We also observe that almost no links are clicked after 2000 in Y coordinates. For most screen sizes, reaching these coordinates requires scrolling down. This means people most of time do not make the effort to explore the whole article before making a choice.

Comparing the plots, we can see that people who found a path clicked more on links between 1000 and 1500 in Y coordinate than those who didn't find a path. This shows that they are more likely to explore the entire article presented to them.

### What next ?

For further analysis, it will be interesting to see how these variables vary over time, i.e. whether people explore articles more at the start of the path than after several clicks.

It will also be interesting to see the distribution of the position of links chosen by people as a function of their performance, i.e. as a function of path size. Are the best paths obtained more often by exploring more of the article?

# 3. Analyse crowd performance vs average performance

Our idea is to start a game with a given source `src` and target `dst` that might not have been played before. We then exploit all the data of the previous games. To choose the second page to click on, we aggregate all the paths that either have source `src` and destination `dst`, or those that go through `src` and have target `dst`. This way, we have the next page each player chose after `src`. We select our next page using majority voting. We repeat this operation until we reach `dst` (we call this procedure the crowd algorithm). For Condorcet's jury theorem to apply, we need to maximise the number of voters at each step. To do this, we chose paths that maximize 'voter scores'. We call the voter score of a path the minimum number of voters encountered by the crowd algorithm. (ie: at each step of the path, we guarantee a certain number of voters)

We run this algorithm on each (`src`, `dst`) tuple with a voter score > 50 and compare the results with what the real players obtained on average for the same (`src`, `dst`) tuple.

The code doing this can be found in crowd.py

The data collected this way is stored in the file `data/promising_crowd_paths.csv`. On these paths, we computed the performance of the crowd and the individual players in the file `data/crowd_vs_players.csv`

Let's see if the crowd outperformed the individual players!

In [30]:
# uncomment the following lines to rerun the crowd algorithm
# import src.crowd as crowd
# crowd.compute_scores(finished_paths_df)
# crowd.stats_players_crowd(finished_paths_df)

# Read CSV file containing the final scores
crowd_res = pd.read_csv("./data/crowd_vs_players.csv")

In [31]:
fig = go.Figure()
fig.add_trace(go.Box(y=crowd_res['players_score'], name='Real Data'))
fig.add_trace(go.Box(y=crowd_res['crowd_score'], name='Crowd Data'))
fig.update_layout(
    title="Comparison of Real Path Lengths and Crowd Path Lengths",
    yaxis_title="Averages",
    xaxis_title="",
)
fig.show()

In [32]:
import plotly.figure_factory as ff

fig = ff.create_distplot(
    [crowd_res['players_score'], crowd_res['crowd_score']], 
    group_labels=['Real Data', 'Crowd Data'], 
    bin_size=1,
    colors=["rgba(0, 0, 255, 0.3)", "rgba(255, 0, 0, 0.3)"]
)
fig.update_layout(
    title="Distribution of averages",
    yaxis_title="Density",
    xaxis_title="Path Length",
    height=800,
)
fig.data = [trace for trace in fig.data if trace.yaxis != 'y2']

fig.show()

Let's see for how many games, the crowd was right or had the same length

In [33]:
print("Number of games where the crowd performed as well of better")
np.sum((crowd_res['players_score'] - crowd_res['crowd_score']) >= 0) / len(crowd_res['players_score'])

Number of games where the crowd performed as well of better


0.9157894736842105

So 91% of time, the crowd was right.

In [34]:
import scipy.stats

# Let's compute a T-Test to see if the averages are significantly different, we choose a a paired t-test. 
# This test compares the means of two related groups
res = scipy.stats.ttest_rel(crowd_res['players_score'], crowd_res['crowd_score'], alternative='less')
print("T-Test result: ", res)
res.pvalue

T-Test result:  TtestResult(statistic=8.616646605369024, pvalue=0.9999999999999201, df=94)


0.9999999999999201

The null hypothesis (="the mean of the distribution underlying ans_stats is greater than the mean of the distribution underlying crowd_stats").

P-value >> 0.05, this indicates that our observation is not so unlikely to have occurred by chance.

Therefore, we do not reject the null hypothesis

# Conclusion

From our analysis of the Wikispeedia dataset, we’ve uncovered valuable insights for everyday decision-making. First, our brains remain more logical and reliable than LLMs, reminding us to trust and engage our critical thinking. Second, shortcuts driven by laziness often hinder progress, emphasizing the importance of effort and thoroughness. Finally, seeking advice from others enriches our understanding and helps us make more informed decisions, though it’s crucial to evaluate input carefully to avoid groupthink.